<a href="https://colab.research.google.com/github/gageswanston/BNFO301_Swanston_Gage/blob/main/BNFO301_Lab7_3_Student_Version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Lab 7. Phylogenetic tree inference using distance matrix methods.

In this three part lab, you will infer a phylogenetic tree using a Distance Matrix Method.  This example uses one of a class of methods, which are statistically consistent, but sensitive to long-branch attraction.  In order to infer a tree, we must take several step, starting with sequences that have been aligned and are of equal length.  Next, we must find all pairwise distances and correct the distances using one of many models of subsitution.  Finally, we need to group sequence pairs and re-calculate distances, until all pairs have been grouped.  This grouping determines the order the tree, represented as a newick string.

##Part 1. 10 points - March 14

Problem 1.1 create a function to return P-distance

Problem 1.2 create a function to return corrected P-distance using the Jukes-Cantor model.

Problem 1.3 create a function to build a pairwise distance matrix

Problem 1.4 create a function to find the pair of sequences with smallest distance, given the distance matrix from part 1.3

##Part 2. 10 points - March 21

Problem 2.1 create a function to merge the most similar sequences based on the corrected P-distances and shrink the matrix

##Part 3. 10 points - March 28

Problem 3.1 create a function to infer a phylogenetic tree

Problem 3.2 create a function to read in a sequence alignment in phylip format

Problem 3.3 use functions created in parts 1 and 2 to infer a phylogenetic tree given the data provided in the phylip file

Problem 3.4 Answer questions about this tree

In [1]:
import math

Problem 1.1 Create function to calculate a P-distance given two sequences

In [2]:
#####################
#
#create a definition called "PDistance" that returns an uncorrected P-distance given two sequences
#your function should first check that sequences are of the same length, else return helpful error message (use keyword assert)
#recall that P-distance is the Hamming-distance / length of the alignment
#
#####################

def PDistance(x, y):
	assert len(x) == len(y), 'sequences must be of equal length'
	counter = 0.0
	for i in range(len(x)):
		if (x[i] != y[i]):
			counter += 1
	Pdist = counter/len(x)
	return Pdist

#####################
#
#use these two sequences to test your function;
#P-distance = 0.2
#
######################

seq1a = "TATAGTAAAA"
seq2a = "TATAGTATTA"
Dist_Problem1_1 = PDistance(seq1a, seq2a)
print(Dist_Problem1_1)

0.2


Problem 1.2. Create a function to correct the P-distance using the Jukes-Cantor model

In [3]:
#####################
#
#create a definition called "JCdistance" that returns a corrected P-distance given two sequences using the Jukes Cantor model
#recal, Jukes Cantor was described in class is a simple correction of P-distance
#Hint: use function created in problem 1.1, "PDistance", to simplfly your code
#
#####################

def JCdistance(x, y):
  uncorrectedDist = PDistance(x, y)
  if (uncorrectedDist != 0):
    correctdist = (-3/4)*math.log(1-(4/3)*uncorrectedDist)
  else:
    correctdist = 0.0
  return (correctdist)

#####################
#
#use these two sequences to test your function;
#Corrected P-distance = 0.2326161962278796
#
######################

seq1b = "TATAGTTAAA"
seq2b = "TATAGTTTTA"
JCdist_Problem1_2 = JCdistance(seq1b, seq2b)
print(JCdist_Problem1_2)

0.2326161962278796


Problem 1.3. Create a function to store distances between sequence pairs as a multidimentionsal dictionary.  You are required to use the Jukes Cantor corrected P-distance when building the dictionary.



In [4]:
#####################
#
#create a definition called "DistanceMatrix" that returns a dictionary given a set of DNA or AA sequences
#Input data: a dictionary containing sequences (values) and sequence names (keys)
#Output data: nested dictionary discribing pairwise distances between sequence pairs. Note: you should include all comparisons, including to self
#
#####################

def DistanceMatrix(sequences):
	matrix = {}
	for i in sequences.keys():
		if i not in matrix.keys():
			matrix[i] = {}
		for j in sequences.keys():
			matrix[i][j] = JCdistance(sequences[i], sequences[j])

	return matrix

#####################
#
#use these sequences to test your function
#expected result: {'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
#
#####################

SampleSeqDict = {'seq1': 'TATACTAAAA', 'seq2': 'TATACTAATT', 'seq3': 'TATAGTATTA', 'seq4': 'TATACTAATT'}
Matrix_Problem1_3 = DistanceMatrix(SampleSeqDict)
print(Matrix_Problem1_3)

{'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}


Problem 1.4. Create a function to find the most similar sequences, given the nested dictionary you created in the last problem.

In [5]:
#####################
#
#create a definition called "GetMinDistance" that returns name of two sequences that has the shortest distance
#Note: avoid self comparisons when finding the shortest distance
#
#####################

def GetMinDistance(matrix):
	lowest_val = float("inf")
	lowest_i = ""
	lowest_j = ""

	for i in matrix.keys():
		for j in matrix.keys():
			if (matrix[i][j] < lowest_val) and (i != j):
				lowest_val = matrix[i][j]
				lowest_i = i
				lowest_j = j

	return lowest_i, lowest_j

#####################
#
#use these sequences to test your function, seq1 and seq2 are most similar within the test data
#Expected result: seq1 and seq2
#
#####################

SampleSeqDict = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Matrix_Problem1_4 = DistanceMatrix(SampleSeqDict)
i, j = GetMinDistance(Matrix_Problem1_4)
print(i,j)

seq1 seq2


Problem 2.1. Create a function to merge the closest neighbors and shrink the nested dictionary created in the previous meeting.

In [6]:
#####################
#
#create a definition called "ShrinkMatrix" that will reduce a nested dictionary
#
#steps to complete to reduce the nested dictionary:
#1. determine pair of labels to be grouped, based on similarity, use 'GetMinDistance' function created above
#2. determine new label for grouped pair - Newick format - (A,B)
#3. generate list of labels not being grouped, set of all labels - set of grouped pair
#4. populate a new nested dictionary with known distances from original nested dictionary
#5. add the new 'grouped taxa' to nested dictionary
#6. populate the nested dictionary with averaged distances
#
#####################
def ShrinkMatrix(matrix):
	#determine pair of labels to be grouped, based on similarity
	name1,name2 = GetMinDistance(matrix)

	#determine new label for grouped pair - Newick format
	newLabel = "({0},{1})".format(name1, name2)

	#generate list of labels not being grouped, set of all labels - set of grouped pair
	namelist = list(set(matrix.keys()).difference(set([name1, name2])))

	#fill in known distances
	nMatrix = {}
	for first in namelist:
		if first not in nMatrix.keys():
			nMatrix[first] = {}
		for second in namelist:
			nMatrix[first][second] = matrix[first][second]

	#add newLabel to nMatrix
	nMatrix[newLabel] = {}

	#fill in averaged distances
	for other in namelist:
		d1 = matrix[other][name1]
		d2 = matrix[other][name2]
		avg = (d1 + d2) / 2.0

		nMatrix[newLabel][other] = avg
		nMatrix[other][newLabel] = avg

	nMatrix[newLabel][newLabel] = 0.0

	return nMatrix


#####################
#
#use these sequences to test your function,
#Expected result: 'seq1' and 'seq2' should be grouped in the new dictionary
#
#####################

SeqDict_Problem2_1 = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Distances_Problem2_1 = DistanceMatrix(SeqDict_Problem2_1)
Matrix_Problem2_1 = ShrinkMatrix(Distances_Problem2_1)
print(Distances_Problem2_1)
print(Matrix_Problem2_1)

{'seq1': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq3': {'seq1': 0.10732563273050497, 'seq2': 0.10732563273050497, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
{'seq3': {'seq3': 0.0, 'seq4': 0.3831192178244929, '(seq1,seq2)': 0.10732563273050497}, 'seq4': {'seq3': 0.3831192178244929, 'seq4': 0.0, '(seq1,seq2)': 0.2326161962278796}, '(seq1,seq2)': {'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796, '(seq1,seq2)': 0.0}}


Problem 3.1. create a function to infer a phylogenetic tree using the distance matrix

In [41]:
#####################
#
#create a definition called "ConstructPhylogeneticTree" that will produce a newick string when provided a distance matrix
#
#####################
def ConstructPhylogeneticTree(distance_matrix): #~
  temp=distance_matrix
  for i in range (len(distance_matrix.keys())-1):
    temp = ShrinkMatrix(temp)
  return(list(temp.keys())[0]+';')








#####################
#
#use these sequences to test your function
#Expected result: (seq4,(seq3,(seq1,seq2)));
#
#####################

SeqDictProblem3_1 = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
DistanceProblem3_1 = DistanceMatrix(SeqDictProblem3_1)
newickStringProblem3_1 = ConstructPhylogeneticTree(DistanceProblem3_1)
print(newickStringProblem3_1)

(seq4,(seq3,(seq1,seq2)));


Please run box below to load phylip formated file with real sequence data

In [23]:
import os.path
# Load the phylip file
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/NADH2_Shark.phy"
DEFAULT_FILE_NAME = 'NADH2_Shark.phy'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB

print("Reading file:", fileName)

# Read in the string from the file
with open(fileName, "r") as myfile:
    file_input = myfile.read()

--2024-03-28 16:52:10--  https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/NADH2_Shark.phy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32765 (32K) [text/plain]
Saving to: ‘NADH2_Shark.phy’

NADH2_Shark.phy     100%[===================>]  32.00K  --.-KB/s    in 0.002s  

2024-03-28 16:52:10 (14.5 MB/s) - ‘NADH2_Shark.phy’ saved [32765/32765]

Reading file: NADH2_Shark.phy


Problem 3.2. Create a function to read in sequence data in phylip format.  Relaxed phylip is a commonly used file format in phylogenetics.  Here you will find sequence name - sequence pairs on each line.

In [38]:
#####################
#
#create a definition called LoadPhylip that will load the real data and create a ditionary of sequence name-sequence key-pairs
#
#####################
def LoadPhylip(filepath):
  myDict={}
  with open (filepath, 'r') as infile:
    for line in infile:
      NnS=line.split()
      myDict[NnS[0]]=NnS[1]
  return(myDict)





#####################
#
#call your definition to build the dictionary
#
#####################
real_sequences = LoadPhylip(DEFAULT_FILE_NAME)


Problem 3.3. Use existing functions to infer a phylogenetic tree using the real data obtained from the phylip formated file.

In [43]:
#####################
#
#use your functions to infer a phylogenetic tree given the data provided in the phylip file
#
#####################
ConstructPhylogeneticTree(DistanceMatrix(LoadPhylip(DEFAULT_FILE_NAME)))







'(U91426.1,(((KF927894.1,JQ519166.1),KF927984.1),((JQ518749.1,KF927801.1),(((JQ519067.1,((JQ519064.1,JQ518745.1),JQ518747.1)),(JQ519066.1,JQ518744.1)),((JQ518743.1,(JQ518751.1,(((JQ519168.1,JQ518741.1),JQ518742.1),KF679782.1))),(((JQ518748.1,(JQ519010.1,((JQ519020.1,(JQ519019.1,((JQ519025.1,JQ519011.1),(KF111729.1,KF928002.1)))),JQ519048.1))),(KF927997.1,KF927998.1)),(JQ518989.1,JQ519055.1)))))));'

Problem 3.4. View tree and answer questions.

*View Tree*. I recomend viewing your tree using an NCBI tool here linked below.  Use the drop down menu on the right hand side to select "Tools/Layout/Slanted Cladogram".  On the left had side is a search box that will allow you to search tips labels.

https://www.ncbi.nlm.nih.gov/projects/treeview/tv.html?appname=ncbi_tviewer&renderer=radial&openuploaddialog

Alternatively, you may use R to view your tree.  The package "APE" will allow you to read, root, and view the tree.  

You will find that the current ordering places tip U91426, so your tree does not require rooting.  

The tip labels in this tree are NCBI genbank identifiers.  You can visit https://www.ncbi.nlm.nih.gov/ and look up the genus and species associated with each tip label.  


*Question 1*. The tip labeled KF679782, represents a sample from *Rhincodon typus*.  Which tip or tips are most closely related to *Rhincodon typus* in your phylogenetic tree?  Please provide your response using the genus and species associated with the genbank identifier.  

*Question 2*. Do you believe the close association between *R. typus* and its closest neighbors in your tree represent the true evolutionary relationship of these species or an erroneus association?  Please explain your answer.

You can use the code space below to respond, simply use "#" to prevent the interpreter from compling your answer as code.



In [ ]:
#((JQ518743.1,(JQ518751.1,(((JQ519168.1,JQ518741.1),JQ518742.1),KF679782.1)))
# In this tree, Rhincodon typus is most closely related to Nebrius ferrugineus, followed by Ginglymostoma cirratum &  Ginglymostoma cf. cirratum
#
# Though determining phylogenetic trees based off of p-values can be error prone, I believe this tree represents the true evolutionary relationship.
# I believe this because ‘Ginglymostoma cirratum’ &  ‘Ginglymostoma cf. cirratum’, the closest two neighbors, share a genus, ‘Nebrius ferrugineus’ shares the Family Ginglymostomatidae with them,
# and all four of them are members of the order Orectolobiformes. This tracks with the order of nodes that we see in the tree.
#
